# 基于GloVe词向量的文本分类
> 作者：宋文彦  
> 学号：21302010062

## 实验目标
本实验将基于GloVe词向量和PyTorch构建一个文本分类模型，使用SST2情感分析数据集进行训练、验证和测试。我们将构建以下模型：
1. **GloVe + Transformer + Pooling + Classifier**
2. **GloVe + RNN + Pooling + Classifier**

## 实验步骤
1. 导入PyTorch和TorchText库，加载SST2数据集
2. 处理数据：文本数据和标签处理，构建词汇表，加载GloVe词向量
3. 定义文本分类模型（基于GloVe + Transformer 和 GloVe + RNN 的模型）
4. 训练模型、验证模型，并评估性能

### Step 1: 导入PyTorch和TorchText库

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import SST2
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torchtext.transforms import VocabTransform, ToTensor
from torch.nn.utils.rnn import pad_sequence

ModuleNotFoundError: No module named 'torch'

### Step 2: 处理数据和标签

在本节中，我们将创建自定义的文本处理器 `TextProcessor` 和标签处理器 `label_processor`。这些处理器将用于对文本进行分词、映射到词汇表索引，并将标签转换为张量。
我们还会定义一个 `collate_fn` 函数，用于在批处理数据时对文本进行填充。

In [ ]:
# 使用 GloVe 词向量
glove = GloVe(name='6B', dim=100)

# 文本处理器
class TextProcessor:
    def __init__(self, vocab, max_length=200):
        self.vocab_transform = VocabTransform(vocab)
        self.max_length = max_length
        self.to_tensor = ToTensor()

    def __call__(self, text):
        # 分词并转换为小写，然后映射到词汇表索引
        tokens = text.lower().split()
        tokens = self.vocab_transform(tokens)
        return self.to_tensor(tokens[:self.max_length])  # 截断到 max_length 并转为张量


In [ ]:
# 标签处理器：将标签转换为张量
def label_processor(label):
    return torch.tensor(int(label))

# 批处理函数
def collate_fn(batch):
    texts, labels = zip(*batch)
    texts = [text_processor(text) for text in texts]
    texts = pad_sequence(texts, batch_first=True)  # 填充到相同长度
    labels = torch.stack([label_processor(label) for label in labels])
    return texts, labels

### Step 3: 加载SST2数据集
使用 `torchtext.datasets.SST2` 来加载Stanford Sentiment Treebank (SST2)数据集，包含训练、验证和测试集。
我们将使用 `DataLoader` 来对数据进行批量处理，并应用我们定义的 `collate_fn`。


In [ ]:
# 加载 SST2 数据集
train_data = list(SST2(split='train'))
valid_data = list(SST2(split='validation'))
test_data = list(SST2(split='test'))

In [ ]:
# 创建 DataLoader
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

### Step 4: 构建模型
本部分将定义文本分类模型，模型采用以下结构：
1. **Embedding Layer**: 采用 GloVe 预训练的词向量初始化嵌入层，词向量不可训练。
2. **Transformer 或 RNN 编码器**：用于提取句子的特征表示。
3. **Pooling Layer**: 自适应最大池化，用于从编码后的序列中提取最重要的特征。
4. **Classifier**: 一个简单的全连接分类器。

#### 4.1 GloVe + Transformer 模型

In [ ]:
class TransformerTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(TransformerTextClassificationModel, self).__init__()
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(glove.vectors)  # 加载 GloVe 词向量
        self.embedding.weight.requires_grad = False  # 不训练词向量

        # Transformer 编码器
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, nhead=2, dim_feedforward=hidden_dim), num_layers
        )
        self.pooling = nn.AdaptiveMaxPool1d(1)  # 自适应最大池化
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        embedded = self.embedding(x)
        encoded = self.transformer_encoder(embedded)
        pooled = self.pooling(encoded.permute(0, 2, 1)).squeeze(-1)  # 池化并去掉多余维度
        return self.classifier(pooled)

#### 4.2 GloVe + RNN 模型

In [ ]:
class RNNTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(RNNTextClassificationModel, self).__init__()
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # RNN 层
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.pooling = nn.AdaptiveMaxPool1d(1)  # 自适应最大池化
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_output, _ = self.rnn(embedded)
        pooled = self.pooling(rnn_output.permute(0, 2, 1)).squeeze(-1)
        return self.classifier(pooled)

### Step 5: 创建模型、优化器和损失函数
我们将使用交叉熵损失函数（`nn.CrossEntropyLoss`）来训练模型，并使用Adam优化器（`optim.Adam`）。

In [ ]:
# 定义模型超参数
vocab_size = len(glove.stoi)
embedding_dim = 100
hidden_dim = 128
num_layers = 2
num_classes = 2  # SST2 是二分类任务

# 创建 Transformer 模型
model = TransformerTextClassificationModel(vocab_size, embedding_dim, hidden_dim, num_layers, num_classes)

# 创建优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

### Step 6: 定义训练和评估函数

In [ ]:
# 训练函数
def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for texts, labels in dataloader:
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

# 评估函数
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for texts, labels in dataloader:
            predictions = model(texts)
            loss = criterion(predictions, labels)
            total_loss += loss.item()
            predicted_labels = predictions.argmax(1)
            correct += (predicted_labels == labels).sum().item()
    return total_loss / len(dataloader), correct / len(dataloader.dataset)


### Step 7: 训练模型
在多个epoch上训练模型，并在每个epoch结束时评估模型在验证集上的表现。

In [ ]:
# 训练模型
N_EPOCHS = 16
for epoch in range(N_EPOCHS):
    train_model(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_loader, criterion)
    print(f'Epoch: {epoch+1:02}, Validation Loss: {valid_loss:.3f}, Validation Acc: {valid_acc*100:.2f}%')

### Step 8: 测试模型性能

In [ ]:
# 在测试集上评估模型
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%')